In [98]:
import numpy as np


def compute_f(A, B, observed):
    n_nodes = len(observed)
    n_states = A.shape[0]
    f = np.zeros((n_nodes - 1, n_states, n_states))

    for i in range(n_nodes - 1):
        tmp = np.zeros((n_states, n_states))
        for j in range(n_states):
            for k in range(n_states):
                if B[k, observed[i + 1]] == 0:
                    tmp_B = np.min(B[B != 0])
                else:
                    tmp_B = np.log(B[k, observed[i + 1]])

                tmp[j, k] = np.log(A[j, k]) + tmp_B
        f[i] = tmp

    return f

In [99]:
A = np.array([[0.3, 0.7], [0.9, 0.1]])
B = np.array([[0.2, 0.8], [0.5, 0.5]])
observed = np.array([0, 1, 1])
compute_f(A, B, observed)

array([[[-1.42711636, -1.04982212],
        [-0.32850407, -2.99573227]],

       [[-1.42711636, -1.04982212],
        [-0.32850407, -2.99573227]]])

In [100]:
def Viterbi(A, B, observed):
    n_nodes = len(observed)
    n_states = A.shape[0]

    pmax = np.zeros((n_nodes - 1, n_states))
    phi = np.zeros((n_nodes - 1, n_states))

    f = compute_f(A, B, observed)

    pmax[0] = np.max((f[0]), axis=0)
    phi[0] = np.argmax(f[0], axis=0)

    for i in range(1, n_nodes - 1):
        tmp = ((f[i]).T + pmax[i - 1]).T

        pmax[i] = np.max(tmp, axis=0)

        phi[i] = np.argmax(tmp, axis=0)

    return pmax, phi

In [65]:
Viterbi(A, B, observed)

(array([[-0.32850407, -1.04982212],
        [-1.37832619, -1.37832619]]),
 array([[1., 0.],
        [1., 0.]]))

In [136]:
def reconstruct(pmax, phi):
    """
    Ricostruisce tutti i most probable, per ora fa schifo ma sembra funzionare, da ricontrollare
    """

    reconstruction = np.empty(len(pmax)+1)
    reconstruction[-1] = np.argmax(pmax[-1])
    curr = np.argmax(pmax[-1])
    for i in range(1, len(pmax)-1):
        reconstruction[-1-i] = phi[-1-i][curr]
        curr = phi[i][curr]
    
    return reconstruction



In [137]:
reconstruct(pmax, phi)

array([0., 0., 1.])

### Experiement with matrix like the one we have (last row of spaces has 1)

In [157]:
def compute_f(A, B, observed):
    """
    Aggiustato il compute_f che dava problemi, ho cambiato observed in i+1 invece che i
    poi questa è l'implementazione senza il log

    Ho anche aggiustato che in realtà il primo messaggio h -> z_1 NON è 0! (guarda formule)
    """
    n_nodes = len(observed)
    n_states = A.shape[0]
    f = np.zeros((n_nodes, n_states, n_states))


    for i in range(n_nodes):
        tmp = np.zeros((n_states, n_states))
        for j in range(n_states):
            for k in range(n_states):
                tmp[j, k] = (A[j, k]) * B[k, observed[i]]
        f[i] = tmp

    return f

In [156]:
def Viterbi(f):
    n_nodes = 3
    n_states = 2

    pmax = np.zeros((n_nodes - 1, n_states))
    phi = np.zeros((n_nodes - 1, n_states))

    pmax[0] = np.max((f[0]), axis=0)
    phi[0] = np.argmax(f[0], axis=0)

    for i in range(1, n_nodes - 1):
        tmp = ((f[i]).T + pmax[i - 1]).T

        pmax[i] = np.max(tmp, axis=0)

        phi[i] = np.argmax(tmp, axis=0)

    return pmax, phi

In [158]:
A = np.array([[0.1, 0.8, 0.1],
              [0.8, 0.1, 0.1],
              [0.5, 0.5, 0]])

B = np.array([[0.5, 0.5, 0],
              [0.5, 0.5, 0],
              [0, 0, 1]])

observed = np.array([2, 0, 1])

In [159]:
f = compute_f(A, B, observed)
print(f)

[[[0.   0.   0.1 ]
  [0.   0.   0.1 ]
  [0.   0.   0.  ]]

 [[0.05 0.4  0.  ]
  [0.4  0.05 0.  ]
  [0.25 0.25 0.  ]]

 [[0.05 0.4  0.  ]
  [0.4  0.05 0.  ]
  [0.25 0.25 0.  ]]]


- Errore in compute_f, parte dalla prima osservazione e invece dovrebbe partire dalla seconda (la prima non contribuisce ai messaggi se non )
- Controlla come viene considerato lo spazio

In [166]:
np.log(0.3) + np.log(0.4)

-2.120263536200091

In [ ]:
np.log(0.3) + np.log(0.4)

In [167]:
2*np.log(0.1) + np.log(0.4)

-5.521460917862246